# Bias on Wikipedia

For this assignment (https://wiki.communitydata.cc/HCDS_(Fall_2017)/Assignments#A2:_Bias_in_data), your job is to analyze what the nature of political articles on Wikipedia - both their existence, and their quality - can tell us about bias in Wikipedia's content.


## Setup

In [1]:
import json
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import requests

%matplotlib inline

data_dir = 'data/'

## Data Ingest

First step is to get the data. We need population data and article data.

### Population Data

Download population data from Population Reference Bureau here:  
http://www.prb.org/DataFinder/Topic/Rankings.aspx?ind=14

This data is saved in `./data/raw/`

In [2]:
# From local data
filename = data_dir + 'raw/Population Mid-2015.csv'
population_data = pd.read_csv(filename, skiprows=2, thousands=',')
population_data.head(4)

,Location,Location Type,TimeFrame,Data Type,Data,Footnotes
0,Afghanistan,Country,Mid-2015,Number,32247000,NaN
1,Albania,Country,Mid-2015,Number,2892000,NaN
2,Algeria,Country,Mid-2015,Number,39948000,NaN
3,Andorra,Country,Mid-2015,Number,78000,NaN


In [3]:
# # From website
# filename = 'http://www.prb.org/RawData.axd?ind=14&fmt=14&tf=76&loc=34235%2c249%2c250%2c251%2c252%2c253%2c254%2c34227%2c255%2c257%2c258%2c259%2c260%2c261%2c262%2c263%2c264%2c265%2c266%2c267%2c268%2c269%2c270%2c271%2c272%2c274%2c275%2c276%2c277%2c278%2c279%2c280%2c281%2c282%2c283%2c284%2c285%2c286%2c287%2c288%2c289%2c290%2c291%2c292%2c294%2c295%2c296%2c297%2c298%2c299%2c300%2c301%2c302%2c304%2c305%2c306%2c307%2c308%2c311%2c312%2c315%2c316%2c317%2c318%2c319%2c320%2c321%2c322%2c324%2c325%2c326%2c327%2c328%2c34234%2c329%2c330%2c331%2c332%2c333%2c334%2c336%2c337%2c338%2c339%2c340%2c342%2c343%2c344%2c345%2c346%2c347%2c348%2c349%2c350%2c351%2c352%2c353%2c354%2c358%2c359%2c360%2c361%2c362%2c363%2c364%2c365%2c366%2c367%2c368%2c369%2c370%2c371%2c372%2c373%2c374%2c375%2c377%2c378%2c379%2c380%2c381%2c382%2c383%2c384%2c385%2c386%2c387%2c388%2c389%2c390%2c392%2c393%2c394%2c395%2c396%2c397%2c398%2c399%2c400%2c401%2c402%2c404%2c405%2c406%2c407%2c408%2c409%2c410%2c411%2c415%2c416%2c417%2c418%2c419%2c420%2c421%2c422%2c423%2c424%2c425%2c427%2c428%2c429%2c430%2c431%2c432%2c433%2c434%2c435%2c437%2c438%2c439%2c440%2c441%2c442%2c443%2c444%2c445%2c446%2c448%2c449%2c450%2c451%2c452%2c453%2c454%2c455%2c456%2c457%2c458%2c459%2c460%2c461%2c462%2c464%2c465%2c466%2c467%2c468%2c469%2c470%2c471%2c472%2c473%2c474%2c475%2c476%2c477%2c478%2c479%2c480'
# population_data = pd.read_csv(filename, skiprows=2, thousands=',')
# population_data.head(4)

In [5]:
# TODO: consider using Gary Gregg's map to update some of the country names

country_map = {
   "East Timorese" : "Timor-Leste",
   "Hondura" : "Honduras",
   "Rhodesian" : "Zimbabwe",
   "Salvadoran" : "El Salvador",
   "Samoan" : "Samoa",
   "São Tomé and Príncipe" : "Sao Tome and Principe",
   # "Somaliland" : "Somalia",  # Oliver says this one is not correct
   "South African Republic" : "South Africa",
   "South Korean" : "Korea, South"
}

OK, so we have the population data by country, both direct from online and from local data. Good.

Now let's see about getting the article data.

### Politician/Article Data

You'll find the wikipedia politician article dataset on Figshare here:  
https://figshare.com/articles/Untitled_Item/5513449

If you want to do this yourself, you'll need to go to the link above, read through the documentation for this repository, then download and unzip it.

In [ ]:
# From website: TBD

In [6]:
# From local data
filename = data_dir + 'raw/page_data.csv'
page_data = pd.read_csv(filename)
page_data.head(4)

,page,country,rev_id
0,Template:ZambiaProvincialMinisters,Zambia,235107991
1,Bir I of Kanem,Chad,355319463
2,Template:Zimbabwe-politician-stub,Zimbabwe,391862046
3,Template:Uganda-politician-stub,Uganda,391862070


## Article Scores from ORES

Now that we have our article data, we can get scores for each article using the ORES API.

Documentation for the ORES API can be found here:  
https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context

In [7]:
endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
headers = {'User-Agent' : 'https://github.com/rexthompson', 'From' : 'rext@uw.edu'}

In [8]:
revids_all = list(page_data['rev_id'])

# TEMP FOR TESTING
revids_all = revids_all[0:500]

In [ ]:
# set up empty dataframe to hold results for each article
df = pd.DataFrame()

# loop 100 entries at a time
idx_start = 0
idx_end = 100
while idx_start < len(revids_all):
    
    # retrieve and concatenate subset of revids
    revids = revids_all[idx_start:idx_end]
    revids = '|'.join(str(x) for x in revids)
    
    # pull article data from API
    params = {'project' : 'enwiki',
              'revids' : revids,
              'model' : 'wp10'
          }
    
    api_call = requests.get(endpoint.format(**params), headers)
    response = api_call.json()
    
    for revid in response['enwiki']['scores']:
        try:
            # temp_dict = response['enwiki']['scores'][revid]['wp10']['score']['probability']
            # rating = max(temp_dict, key=temp_dict.get)
            rating = response['enwiki']['scores'][revid]['wp10']['score']['prediction']
        except:
            rating = np.nan
        df = df.append({'revid':revid, 'rating':rating}, ignore_index=True)
    
    # NOTE -- ratings do not return in the same order as they were passed to the API!!!
    
    # update indexes
    idx_start += 100
    idx_end = min(idx_start+100, len(revids_all))
    
# revid, temp_dict, rating

Let's see how this data looks.

In [ ]:
df.head()

OK, so now we have a nice table of ratings for each article. Let's merge this back with the original article data. First we convert the revids to ints since they are currently strings. We need them to be the same as page_data which has this as an int.

In [ ]:
df['revid'] = pd.to_numeric(df['revid'], errors='coerce')

Then we merge, and drop the redundant column.

In [ ]:
rating_data = page_data.merge(df, left_on='rev_id', right_on='revid').drop('revid', 1)
rating_data.head()

In [ ]:
population_data.head()

Now we'll merge both datasets on the Country column (or the "Location" column for the population data).

In [ ]:
merged_df = population_data.merge(rating_data, left_on='Location', right_on='country')
merged_df_new = pd.DataFrame({'country':merged_df['Location'],
                              'population':merged_df['Data'],
                              'article_name':merged_df['page'],
                              'revision_id':merged_df['rev_id'],
                              'article_quality':merged_df['rating']})

#convert population to int
pd.to_numeric(merged_df_new['population'])

#reorder columns
merged_df_new = merged_df_new[['country',
                               'population',
                               'arrticle_name',
                               'revision_id',
                               'article_quality']]
merged_df_new

Let's save this data to CSV.

In [ ]:
# set filename for combined data CSV
filename = data_dir + 'merged_data.csv'

# check if file already exists; load if so, create if not
if os.path.isfile(filename):
    merged_df_new = pd.read_csv(filename)
    print('loaded CSV data from ' + filename)
else:
    merged_df_new.to_csv(filename, index=False)
    print('saved CSV data to ' + filename)

## Analysis

So, now we have a good database with ratings for each article. We can now group by country to count how many of each type there are.

In [ ]:
merged_df_new.head()

In [ ]:
pd.DataFrame(merged_df_new.groupby(['country']).count())

In [ ]:
# The API can only handle so many requests at a time, so we'll go 100 at a time.


# loop over 
params = {'project' : 'enwiki',
          'revids' : '797882322',
          'model' : 'wp10'
          }

api_call = requests.get(endpoint.format(**params), headers)
response = api_call.json()
response
#print(json.dumps(response, indent=4, sort_keys=True))

for revid in response['enwiki']['scores']:
    print(revid)
    temp_dict = response['enwiki']['scores'][revid]['wp10']['score']['probability']
    rating = max(temp_dict, key=temp_dict.get)
    print(rating)

In [ ]:
response

Importing the other data is just a matter of reading CSV files in! (and for the R programmers - we'll have an R example up as soon as the Hub supports the language).

In [ ]:
## getting the data from the CSV files
import csv

data = []
with open('page_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append([row[0],row[1],row[2]])

In [ ]:
print(data[782])

### Note from Andrew Enfield on Slack on 10/25/17:

FYI that I was able to get scores for all articles in just two minutes with the https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context API. I retrieved the scores in chunks of 140 articles at a time/per call - when I experimented, 140 always worked but 145 or 150 gave me errors.

# OLD STUFF

# Bias on Wikipedia

For this assignment (https://wiki.communitydata.cc/HCDS_(Fall_2017)/Assignments#A2:_Bias_in_data), your job is to analyze what the nature of political articles on Wikipedia - both their existence, and their quality - can tell us about bias in Wikipedia's content.

# Making ORES requests

Below is an example of how to make requests through the ORES system in Python to find out the current quality of an article. Specifically, this is a function designed to make a request with *multiple* revision IDs. You can take this function, split your revision IDs up into chunks of 50 or 100 to avoid hitting limits in ORES, pass each chunk through this function, and then stitch the whole set together.

In [ ]:
import requests
import json

headers = {'User-Agent' : 'https://github.com/your_github_username', 'From' : 'your_uw_email@uw.edu'}

def get_ores_data(revision_ids, headers):
    
    # Define the endpoint
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
    
    # Specify the parameters - smushing all the revision IDs together separated by | marks.
    # Yes, 'smush' is a technical term, trust me I'm a scientist.
    # What do you mean "but people trusting scientists regularly goes horribly wrong" who taught you tha- oh.  
    params = {'project' : 'enwiki',
              'model'   : 'wp10',
              'revids'  : '|'.join(str(x) for x in revision_ids)
              }
    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    print(json.dumps(response, indent=4, sort_keys=True))


# So if we grab some example revision IDs and turn them into a list and then call get_ores_data...
example_ids = [783381498, 807355596, 757539710]
get_ores_data(example_ids, headers)

Importing the other data is just a matter of reading CSV files in! And if you're an R programmer wondering where the R example is - check the other file in this example.

In [ ]:
## getting the data from the CSV files
import csv

data = []
with open('./data/raw/page_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append([row[0],row[1],row[2]])

In [ ]:
print(data[782])

# ORIGINAL EXAMPLE

# Get ORES Scores

Below is an example of how to make a request through the ORES system in Python to find out the current quality of the article on [Aaron Halfaker](https://en.wikipedia.org/wiki/Aaron_Halfaker) (the person who created ORES):

Actually use the following link for documentation:  
https://ores.wikimedia.org/v3/#!/scoring/get_v3_scores_context

In [ ]:
endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/{revid}/{model}'
endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'
headers = {'User-Agent' : 'https://github.com/your_github_username', 'From' : 'your_uw_email@uw.edu'}

params = {'project' : 'enwiki',
          'revids' : '797882120|797882121',
          'model' : 'wp10'
          }

api_call = requests.get(endpoint.format(**params))
response = api_call.json()
print(json.dumps(response, indent=4, sort_keys=True))


Importing the other data is just a matter of reading CSV files in! (and for the R programmers - we'll have an R example up as soon as the Hub supports the language).

In [ ]:
## getting the data from the CSV files
import csv

data = []
with open('page_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append([row[0],row[1],row[2]])

In [ ]:
print(data[782])